# Triplet network (ESLR Recruitment Project 2019)

## To-Do

1. Bonus: create a TF Dataset + don't load in RAM

## Resources

- https://arxiv.org/pdf/1412.6622.pdf
- https://omoindrot.github.io/triplet-loss
- https://www.coursera.org/lecture/convolutional-neural-networks/triplet-loss-HuUtN

## Triplet networks

### Definitions

The aim of the triplet loss is to make sure that:
- Two feature vectors with the same label have their embeddings close together in the embedding space.
- Two feature vectors with different labels have their embeddings far away.

To train the neural network we use triplets composed of:
- an anchor
- a positive with the same label as the anchor
- a negative with a different label

The loss function for a triplet (A, P, N) is:  
$ L = max(d(A, P) - d(A, N) + margin, 0) $ 

### Triplet Mining

The real trouble when implementing triplet loss or contrastive loss in TensorFlow is how to sample the triplets or pairs.

1. Offline triplet mining : find them at the beginning of each epoch
2. Online triplet mining : compute useful triplets on the fly

### Implementations

1. https://www.tensorflow.org/addons/tutorials/losses_triplet
2. From scratch: https://stackoverflow.com/questions/38260113/implementing-contrastive-loss-and-triplet-loss-in-tensorflow/38270293#38270293


## Link Google Drive to retrieve the dataset

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Import libraries

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import io
import numpy as np

try:
  %tensorflow_version 2.x
except:
  pass

import tensorflow as tf
!pip install -q  --no-deps tensorflow-addons
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

print(tf.__version__)

TensorFlow 2.x selected.
     |████████████████████████████████| 1.8MB 2.8MB/s 
2.1.0-rc1


## Load dataset

In [0]:
# Load feature vectors
vectors = np.fromfile('/content/drive/My Drive/ember-dataset/Xtrain.dat', dtype=np.float32)
vectors = np.reshape(vectors, (-1, 2351)) # Create a 2d np array

# Load labels
labels = np.fromfile('/content/drive/My Drive/ember-dataset/Ytrain.dat', dtype=np.float32)

# Remove unlabeled vectors
unlabeled_index = np.argwhere(labels==-1).flatten()
labels = np.delete(labels, unlabeled_index, 0)
vectors = np.delete(vectors, unlabeled_index, 0)

print ("Vectors shape:", vectors.shape)
print ("Labels shape:", labels.shape)

Vectors shape: (600000, 2351)
Labels shape: (600000,)


## Split dataset (train + test)

In [0]:
# Split the dataset in two
vectors_splitted = np.split(vectors, [500000])
labels_splitted = np.split(labels, [500000])

# Create a training dataset
train_vectors = vectors_splitted[0]
train_labels = labels_splitted[0]
print ("Train vectors shape:", train_vectors.shape)
print ("Train labels shape:", train_labels.shape)

# Create a test dataset
test_vectors = vectors_splitted[1]
test_labels = labels_splitted[1]
print ("Test vectors shape:", test_vectors.shape)
print ("Test labels shape:", test_labels.shape)

Train vectors shape: (500000, 2351)
Train labels shape: (500000,)
Test vectors shape: (100000, 2351)
Test labels shape: (100000,)


## Create neural network

Here we use the triplet semi-hard loss function from tensorflow_addons: https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/contrib/losses/metric_learning/triplet_semihard_loss.

This function relies on semi-hard online mining.
As a reminder:
- semi-hard triplets: triplets for which the positive is closer to the anchor than the negative, but which still have loss.
- online mining: compute useful triplets on the fly.

We adapt our network's model by:
1. Removing our output layer (we want embeddings at the end).
2. Removing the activation function of the last layer.
3. L2 normalizing the embeddings. 

In [0]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(256, activation='relu'),

  # No activation on final dense layer (previously relu)
  tf.keras.layers.Dense(256, activation=None),
  
  # No ouput layer: keras.layers.Dense(2, activation='softmax')

  # L2 normalize embeddings
  tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) 
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tfa.losses.TripletSemiHardLoss())

## Train the neural network

Unfortunately, the loss does not seem to decrease over the training period even after trying multiple model topologies. The minimum we got is $\simeq$ 0.85. Not using "batches" from the dataset may explain this phenomenon.

In [0]:
# Train the network
history = model.fit(train_vectors, train_labels, epochs=5)

Train on 500000 samples
Epoch 1/5
500000/500000 [==============================] - 78s 155us/sample - loss: 0.9837
Epoch 2/5
500000/500000 [==============================] - 77s 153us/sample - loss: 0.9057
Epoch 3/5
500000/500000 [==============================] - 76s 153us/sample - loss: 0.8606
Epoch 4/5
500000/500000 [==============================] - 77s 155us/sample - loss: 0.8583
Epoch 5/5
500000/500000 [==============================] - 77s 155us/sample - loss: 0.8537


## Evaluate results

In [0]:
# Evaluate the network
results = model.predict(test_vectors)
print(results.shape)

(100000, 256)


## Use results for k-means

By saving `test_vectors` and `test_labels` as binary files we are able to use them to train our k-means algorithm.

To start k-means using this new dataset some parameters have to be changed: 
`./kmeans 2 20 1.0 256 100000 ../ember/vectors.dat out.dat`. The training is relatively quick because of the small dimension (`256`) and the small number of vectors (`10000`). **Furthermore, the accuracy is of `0.8601`**. 

It is significantly greater than the one we get with the whole dataset. This can be explained by the fact that we train and test on the same dataset but still the result is worthwhile as doing so on the whole dataset would not give such accuracy. Doing more experiemnts would confirm this hypothesis.

Therefore, triplet networks seems to be very efficient to compute new feature vectors (embeddings) which, with their small dimension, can be used in parallel of other classification techniques (machine learning classifiers, deep neural networks, k-means algorithm, ...).

In [0]:
# Export results for k-means
results.flatten().tofile("vectors.dat")
test_labels.tofile("labels.dat")

(25600000,)
(100000,)


## Results projection visualization

**In order to download files quickly, run the cell below if length of results is 10000 not 100000.**

In [0]:
# Export results for visualization in projector
np.savetxt("vectors.tsv", results, delimiter='\t')
out_m = io.open('labels.tsv', 'w', encoding='utf-8')
for label in test_labels:
    out_m.write(str(label) + "\n")
out_m.close()

# vectors.tsv : 256 vectors on 10000 lines
# labels.tsv : one label (0.0 or 1.0) on 10000 lines

We can save results to use them on https://projector.tensorflow.org/. The screenshot below represents 10000 plotted using UMAP (Uniform Manifold Approximation and Projection) for dimension reduction.

As a reminder, **0** represents benign programs and **1** is for malicious programs. We can notice that most of the lines belongs either to the first or to the second class of programs.

![screenshot](https://i.imgur.com/TSGiXfk.png)